In [1]:
pip install accelerate ffmpeg indexify


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from indexify import IndexifyClient
client = IndexifyClient()

In [20]:
import urllib.request

urllib.request.urlretrieve(filename="ALLIN-E167.mp3", url="https://content.libsyn.com/p/5/d/f/5df17f8350f43745/ALLIN-E167.mp3?c_id=168165938&cs_id=168165938&destination_id=1928300&response-content-type=audio%2Fmpeg&Expires=1708908562&Signature=aLXFc-TXV3rb53Kg4ntr05zuHq3b5~6c03CPK4HNfhAwwbHbXf59Wan3NG2p0AzOfHWSqmK4A7sEarPdvi89hBMljCOpL~8HvX8aoGSmktgqkghQ4TDl~Q89DZ1gkuFHQ6BlIK5qAcdMA0NyfDhF5JN6UjI1Ja~OHuB6LjP-lZojmQddvfSdiI4oOJuODxQjffwZS1AaBO0z4yB4EcIqzwe46UWEe4ajL3mpgek9rb3ByyYarrewKq7-ZzpaKLFY6SN7cN2KMOPdyMvGw0rzNM9NfApLhyyEwDmDVOIhmoiqu2Act8U-1Oqcu23Hdm0MziL9ILPmRomu0XBfe9au1w__&Key-Pair-Id=K1YS7LZGUP96OI")

('ALLIN-E167.mp3', <http.client.HTTPMessage at 0x12a06b6e0>)

In [21]:
client.upload_file(path="ALLIN-E167.mp3")

In [22]:
client.add_extraction_policy(extractor='tensorlake/whisper-asr', name="audio-transcription")

In [23]:
client.add_extraction_policy(extractor='tensorlake/minilm-l6', name="transcription-embedding", content_source='audio-transcription')

In [4]:
from indexify_langchain import IndexifyRetriever

In [5]:
params = {"name": "transcription-embedding.embedding", "top_k": 50}
retriever = IndexifyRetriever(client=client, params=params)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key="")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [14]:
chain.invoke("What is Chamath's firm called?")

"Chamath's firm is called Social Capital."